<a href="https://colab.research.google.com/github/victoruwazurike1/Hamoye_Internship_Notebooks/blob/main/hamoye_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import numpy as np
import sklearn.utils
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE 

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = [14,10] # This is to set the configuration of the plots we will create in this analysis

In [2]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         72186 non-null  object 
 1   year            72186 non-null  int64  
 2   country_code    72186 non-null  int64  
 3   record          72186 non-null  object 
 4   crop_land       51714 non-null  float64
 5   grazing_land    51714 non-null  float64
 6   forest_land     51714 non-null  object 
 7   fishing_ground  51713 non-null  float64
 8   built_up_land   51713 non-null  float64
 9   carbon          51713 non-null  float64
 10  total           72177 non-null  float64
 11  QScore          72185 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 6.6+ MB


In [7]:
# Let us check the distribution of target variable
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

We can see that the target column is highly imbalanced

In [8]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [10]:
# For simplicity we aaare going to drop the missing values
df = df.dropna()

In [11]:
# We can now check again to confirm the null values have been dropped
df.isna().sum()


country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [12]:
# We have confirmed we don't have any null values
# Now we will check how many classes our target variable has 
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [13]:
# We see that we are left with 3 classes but still highly imbalanced
# Let us convert this to a binary classification problem by converting class '1A' and '2A'
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' )


In [14]:
# Now we have just 2 classes left
df['QScore'].value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [23]:
# We are going to undersample class '3A'
df_2A = df[df.QScore== '2A' ]
df_3A = df[df.QScore== '3A' ].sample(350)
data_df = df_2A.append(df_3A) 

In [24]:
data_df['QScore'].value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [25]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590 entries, 1536 to 48894
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         590 non-null    object 
 1   year            590 non-null    int64  
 2   country_code    590 non-null    int64  
 3   record          590 non-null    object 
 4   crop_land       590 non-null    float64
 5   grazing_land    590 non-null    float64
 6   forest_land     590 non-null    object 
 7   fishing_ground  590 non-null    float64
 8   built_up_land   590 non-null    float64
 9   carbon          590 non-null    float64
 10  total           590 non-null    float64
 11  QScore          590 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 59.9+ KB


In [26]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop = True)
print(data_df.shape)
data_df.QScore.value_counts()

(590, 12)


3A    350
2A    240
Name: QScore, dtype: int64

In [27]:
data_df = data_df.drop(columns = ['country_code', 'country', 'year'])

In [28]:
X = data_df.drop(columns = 'QScore')
y = data_df['QScore']

In [30]:
# Now let us spit the data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
y_train.value_counts()

3A    242
2A    171
Name: QScore, dtype: int64

We can see that there is still imbalance in the class distribution so we are going to use SMOTE only on the training data to handle this.

In [32]:
# We are going to encode the categorical variables
encoder = LabelEncoder() 
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.fit_transform(x_test.record)


In [54]:
smote = SMOTE(random_state = 1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)

In [47]:
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns = 'record'))
normalised_train_df = pd.DataFrame(normalised_train_df,
columns=x_train_balanced.drop(columns=[ 'record' ]).columns) 
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ]

In [48]:
x_test = x_test.reset_index(drop= True )
normalised_test_df = scaler.transform(x_test.drop(columns=[ 'record' ]))
normalised_test_df = pd.DataFrame(normalised_test_df,
columns=x_test.drop(columns=[ 'record' ]).columns)
normalised_test_df[ 'record' ] = x_test[ 'record']

In [49]:
# Now we can use Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
log_reg.predict(normalised_test_df)

array(['3A', '2A', '2A', '2A', '2A', '3A', '2A', '3A', '2A', '2A', '2A',
       '2A', '3A', '2A', '3A', '2A', '2A', '3A', '2A', '3A', '2A', '2A',
       '2A', '3A', '3A', '2A', '2A', '2A', '2A', '3A', '2A', '2A', '2A',
       '2A', '2A', '2A', '2A', '2A', '3A', '3A', '3A', '2A', '2A', '2A',
       '2A', '2A', '2A', '2A', '2A', '3A', '2A', '2A', '2A', '3A', '3A',
       '2A', '2A', '2A', '2A', '3A', '3A', '2A', '2A', '3A', '2A', '3A',
       '2A', '2A', '3A', '3A', '2A', '3A', '2A', '3A', '2A', '3A', '2A',
       '2A', '3A', '3A', '2A', '2A', '2A', '2A', '3A', '2A', '3A', '3A',
       '2A', '2A', '3A', '2A', '2A', '2A', '3A', '3A', '2A', '2A', '3A',
       '3A', '2A', '2A', '3A', '3A', '3A', '2A', '2A', '3A', '2A', '2A',
       '3A', '3A', '3A', '2A', '3A', '2A', '3A', '2A', '2A', '3A', '2A',
       '3A', '3A', '2A', '2A', '2A', '2A', '2A', '3A', '2A', '2A', '3A',
       '2A', '2A', '3A', '3A', '3A', '2A', '2A', '3A', '2A', '2A', '2A',
       '3A', '2A', '2A', '3A', '2A', '2A', '3A', '3

Measuring classification performance

In [52]:
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv= 5 , scoring= 'f1_macro' )
scores 

array([0.48183761, 0.48404255, 0.56325043, 0.56832736, 0.54811166])

In [58]:
# K fold Cross Validation
kf = KFold(n_splits= 5)
kf.split(normalised_train_df)
f1_scores = []

# We are going to iterate through every split to get the f1_scores

for train_index, test_index in kf.split(normalised_train_df): 
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
  y_train, y_test = y_balanced[train_index], y_balanced[test_index] 
  model = LogisticRegression().fit(x_train, y_train)
  f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), 
                            pos_label= '2A' )* 100 ) 
f1_scores

[50.943396226415096, 48.42105263157895, 53.333333333333336, 54.0, 0.0]